# 1.0 Load CRV2 GEX Data

MTX -> Sparse in-memory Format -> DataFrame (dense)

In [10]:
from clustergrammer2 import net

>> clustergrammer2 backend version 0.5.2


In [1]:
import pandas as pd
import helper_functions_10x as hf

from typing import Dict

# explicitly declare we're using a dictionary of DataFrames
df:Dict[str,pd.DataFrame] = {}

### Load GEX Data into DataFrame

In [3]:
feature_data = hf.load_crv2_gene_matrix('../data/pbmc3k_filtered_gene_bc_matrices_v2-cr/')

In [4]:
hf.check_feature_data_size(feature_data)

gex
32738 2700
(32738, 2700) 



In [5]:
df = hf.convert_feature_data_to_df_dict(feature_data)

In [6]:
df.keys()

dict_keys(['gex'])

In [7]:
df['gex'].shape

(32738, 2700)

In [8]:
df['gex'].head()

,AAACATACAACCAC,AAACATTGAGCTAC,AAACATTGATCAGC,AAACCGTGCTTCCG,AAACCGTGTATGCG,AAACGCACTGGTAC,AAACGCTGACCAGT,AAACGCTGGTTCTT,AAACGCTGTAGCCA,AAACGCTGTTTCTG,...,TTTCAGTGTCACGA,TTTCAGTGTCTATC,TTTCAGTGTGCAGT,TTTCCAGAGGTGAG,TTTCGAACACCTGA,TTTCGAACTCTCAT,TTTCTACTGAGGCA,TTTCTACTTCCTCG,TTTGCATGAGAGGC,TTTGCATGCCTCAC
MIR1302-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM138A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OR4F5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RP11-34P13.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RP11-34P13.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Visualize

In [11]:
net.load_df(df['gex'])
net.filter_N_top(inst_rc='row', N_top=100, rank_type='var')
net.normalize(axis='row', norm_type='zscore')
net.clip(-5,5)
net.widget()

ExampleWidget(network='{"row_nodes": [{"name": "FTL", "ini": 100, "clust": 82, "rank": 85, "rankvar": 83, "gro…